# 导入库函数

In [1]:
from sklearn.cluster import KMeans
import torch
from torch.nn import Embedding
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.nn import Parameter
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, average_precision_score
from sklearn import metrics
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

# evaluating indicator

In [2]:
def predict_acc(recons_edges, true_edges):
    predict_graph = recons_edges
    predict_edges = np.array(predict_graph)
    
    ap = average_precision_score(true_edges, predict_edges)
    print("AP： ", ap)
    fpr, tpr, _ = metrics.roc_curve(true_edges, predict_edges, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    print("AUC SCORE: ",auc)
    
    return ap, auc
    #predict_edges[predict_edges>gama] = 1
    #predict_edges[predict_edges<=gama] = 0
    
    #print("ACC： ",accuracy_score(true_edges, predict_edges))
    #print("AP： ",precision_score(true_edges, predict_edges, average='macro'))
    #print("RECALL： ",recall_score(true_edges, predict_edges, average='macro'))
    #print("F1 SCORE： ",f1_score(true_edges, predict_edges, average='macro'))

# model

In [3]:
"""  Given the cluster centroids, update the node representation  """
class reconstruction_graph(nn.Module):
    def __init__(self, NE, alpha=1.0):
        super(reconstruction_graph, self).__init__()
        self.alpha = alpha
        self.nodes_embedding = Parameter(NE)
        
    def forward(self, CC):
        # Calculate the attention score (in nodes) for each node to each cluster
        norm_squared = torch.sum((self.nodes_embedding.unsqueeze(1) - CC)**2, 2)   # Distance from the node to the center of mass
        numerator = 1.0 / (1.0 + (norm_squared / DEC))
        power = float(self.alpha + 1) / 2   
        numerator = numerator**power    
        soft_assignments = (numerator.t() / torch.sum(numerator, 1)).t()       #soft assignment using t-distribution
        
        # Calculate the cluster similarity between nodes (cosine similarity): the larger the value, the more conducive to edge formation
        prod = torch.mm(soft_assignments, soft_assignments.t())       #  numerator
        norm = torch.norm(soft_assignments,p=2,dim=1).unsqueeze(0)      #    denominator 
        clusters_similar = prod.div(torch.mm(norm.t(),norm))
        
        # Calculate the distance between nodes: smaller values favor edge formation
        nodes_distance = torch.norm(self.nodes_embedding[:, None]-self.nodes_embedding, dim=2, p=2)
        nodes_distance = torch.div(nodes_distance, torch.max(nodes_distance))
        
        # The formation probability of the edges is calculated
        distance_similar = torch.div(beta*nodes_distance, clusters_similar)      
        nodes_similar = torch.exp(-distance_similar)
        
        return nodes_similar

class update_nodes_embedding(nn.Module):
    def __init__(self, ne):
        super(update_nodes_embedding, self).__init__()
        NE = ne
        self.reconstruction_module = reconstruction_graph(NE)     
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=0.9)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, g, CC, edge_train, edge_test):
        self.reconstruction_module.train()
        for epoch in range(5):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(CC)
            graph_train = torch.take(graph_reconstruction, edge_train)
            loss = self.loss_function(g, graph_train)
            loss.backward()
            self.optimizer.step()
            #print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test).detach()
        ap, auc = predict_acc(recons_test_edges, test_edge)
        return self.reconstruction_module.nodes_embedding.detach(), ap, auc
    
"""  Type-centroid update aimed at maximizing the prediction effect   """
class ClusteringLayer(nn.Module):
    """  Given the node representation, update the cluster centroids  """
    def __init__(self, cc, alpha=1.0):
        super(ClusteringLayer, self).__init__()
        self.alpha = alpha
        self.cluster_centers = Parameter(cc)
    
    def forward(self, NE):
        norm_squared = torch.sum((NE.unsqueeze(1) - self.cluster_centers)**2, 2) 
        numerator = 1.0 / (1.0 + (norm_squared / DEC))
        power = float(self.alpha + 1) / 2   
        numerator = numerator**power   
        soft_assignments = (numerator.t() / torch.sum(numerator, 1)).t() 
        
        prod = torch.mm(soft_assignments, soft_assignments.t())
        norm = torch.norm(soft_assignments,p=2,dim=1).unsqueeze(0)
        clusters_similar = prod.div(torch.mm(norm.t(),norm))
        
        nodes_distance = torch.norm(NE[:, None]-NE, dim=2, p=2)
        nodes_distance = torch.div(nodes_distance, torch.max(nodes_distance))
        
        distance_similar = torch.div(beta*nodes_distance, clusters_similar)      
        nodes_similar = torch.exp(-distance_similar)
        
        return nodes_similar

def find_cluster_centers(NE, CC, g, edge_train, edge_test, num_epochs=5):        
    clusteringlayer = ClusteringLayer(CC)
    optimizer = torch.optim.SGD(params=clusteringlayer.parameters(), lr=LR, momentum=0.9)
    loss_function = torch.nn.MSELoss(reduction='sum')
    
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        graph_recons = clusteringlayer(NE)
        graph_train = torch.take(graph_recons, edge_train)
        loss = loss_function(g, graph_train)
        loss.backward()
        optimizer.step()
        #print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')     
    recons_test_edges = torch.take(graph_recons, edge_test).detach()
    ap, auc = predict_acc(recons_test_edges, test_edge)
    return clusteringlayer.cluster_centers.detach(), ap, auc

In [4]:
def get_graph(nodes_number):
    edges = pd.read_csv('datasets/texas.csv', header=None).iloc[1:,]
    edges = edges.astype('int')
    
    graph_np = np.zeros((nodes_number, nodes_number))
    for i in range(edges.shape[0]):
        graph_np[edges.iloc[i, 0], edges.iloc[i, 1]]=1
        graph_np[edges.iloc[i, 1], edges.iloc[i, 0]]=1

    posi_edge = np.argwhere(graph_np == 1)
    posi_edge = posi_edge[posi_edge[:,0]<posi_edge[:,1]]         # Only the upper-left corner matrix is taken
    posi_edge_number = posi_edge.shape[0]
    
    nega_edge = np.argwhere(graph_np == 0) 
    nega_edge = nega_edge[nega_edge[:,0]<nega_edge[:,1]]     # Only the upper-left corner matrix is taken
    
    positive_index = np.random.choice(range(posi_edge_number),int(posi_edge_number*0.9),replace=False)
    choose_positive = posi_edge[positive_index]
    posi_not_choose = np.setdiff1d(range(posi_edge_number), positive_index)
    test_positive = posi_edge[posi_not_choose]
    
    negative_index = np.random.choice(range(nega_edge.shape[0]),int(posi_edge_number*0.9)*6,replace=False)
    choose_negative = nega_edge[negative_index]
    nega_not_choose = np.setdiff1d(range(nega_edge.shape[0]), negative_index)
    test_nega_index = np.random.choice(nega_not_choose,len(posi_not_choose),replace=False)
    test_negative = nega_edge[test_nega_index]
    
    train_posi = [list(choose_positive[i]) for i in range(len(choose_positive))]
    train_nega = [list(choose_negative[i]) for i in range(len(choose_negative))]
    train_index = train_posi + train_nega
    train_mask = [train_index[i][0]*nodes_number+train_index[i][1] for i in range(len(train_index))]
    train_mask = torch.tensor(train_mask)
    
    test_posi = [list(test_positive[i]) for i in range(len(test_positive))]
    test_nega = [list(test_negative[i]) for i in range(len(test_negative))]
    test_index = test_posi + test_nega
    test_mask = [test_index[i][0]*nodes_number+test_index[i][1] for i in range(len(test_index))]
    test_mask = torch.tensor(test_mask)
    
    graph_tensor = torch.from_numpy(graph_np).float()
    train_edge = torch.take(graph_tensor, train_mask)
    test_edge = np.array(torch.take(graph_tensor, test_mask))
    
    return train_edge, test_edge, train_mask, test_mask

In [5]:
nodes_number = 183

In [6]:
embedding_dim = 12
n_clusters = 24
alpha = 1
beta = 4.2
DEC = 4.8
LR = 0.1

all_ap_first_ap = []
all_ap_first_auc = []

all_auc_first_ap = []
all_auc_first_auc = []

for i in range(10):
    train_edge, test_edge, train_mask, test_mask = get_graph(nodes_number)
    ini_embedding = Embedding(nodes_number, embedding_dim, sparse=True)       
    raw_nodes_embedding = ini_embedding.weight.detach()        # Initial node embedding

    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(raw_nodes_embedding)
    cluster_centers = kmeans.cluster_centers_
    raw_cluster_centers = torch.tensor(cluster_centers, dtype=torch.float)     # Initial cluster centroids
    
    ap_first__ap = 0
    ap_first_auc = 0
    auc_first__ap = 0
    auc_first_auc = 0
    for module_epoch in range(300):
        print("######################### Module cycle ： %d ##########################"%module_epoch)
        update_nodes_module = update_nodes_embedding(raw_nodes_embedding)
        raw_nodes_embedding, nodes_ap, nodes_auc = update_nodes_module(train_edge, raw_cluster_centers, train_mask, test_mask)
        if nodes_ap > ap_first__ap:
            ap_first__ap = nodes_ap
            ap_first_auc = nodes_auc
        if nodes_auc > auc_first_auc:
            auc_first_auc = nodes_auc
            auc_first__ap = nodes_ap
        raw_cluster_centers, cluster_ap, cluster_auc = find_cluster_centers(raw_nodes_embedding, raw_cluster_centers, train_edge, train_mask, test_mask)
        if cluster_ap > ap_first__ap:
            ap_first__ap = cluster_ap
            ap_first_auc = cluster_auc
        if cluster_auc > auc_first_auc:
            auc_first_auc = cluster_auc
            auc_first__ap = cluster_ap
            
    all_ap_first_ap.append(ap_first__ap)
    all_ap_first_auc.append(ap_first_auc)
    all_auc_first_ap.append(auc_first__ap)
    all_auc_first_auc.append(auc_first_auc)

######################### Module cycle ： 0 ##########################
AP：  0.6123820574304746
AUC SCORE:  0.6020408163265306
AP：  0.6089519491696929
AUC SCORE:  0.5956632653061225
######################### Module cycle ： 1 ##########################
AP：  0.6465182817165374
AUC SCORE:  0.6364795918367345
AP：  0.6562292413847228
AUC SCORE:  0.6517857142857143
######################### Module cycle ： 2 ##########################
AP：  0.666828799344355
AUC SCORE:  0.6645408163265306
AP：  0.6926236030967619
AUC SCORE:  0.6772959183673469
######################### Module cycle ： 3 ##########################
AP：  0.7008144331632689
AUC SCORE:  0.6913265306122449
AP：  0.6984361117299962
AUC SCORE:  0.6913265306122449
######################### Module cycle ： 4 ##########################
AP：  0.7118754501292166
AUC SCORE:  0.6913265306122449
AP：  0.7145667562142833
AUC SCORE:  0.6977040816326532
######################### Module cycle ： 5 ##########################
AP：  0.7161190163999451
AUC SCO

AP：  0.6625372691297459
AUC SCORE:  0.6658163265306123
######################### Module cycle ： 47 ##########################
AP：  0.66369016931421
AUC SCORE:  0.6683673469387755
AP：  0.663842794466835
AUC SCORE:  0.6670918367346939
######################### Module cycle ： 48 ##########################
AP：  0.6633335211314748
AUC SCORE:  0.667091836734694
AP：  0.6640175649102766
AUC SCORE:  0.6683673469387755
######################### Module cycle ： 49 ##########################
AP：  0.665331442455054
AUC SCORE:  0.6709183673469388
AP：  0.6652273221091956
AUC SCORE:  0.6721938775510204
######################### Module cycle ： 50 ##########################
AP：  0.6542016810835546
AUC SCORE:  0.6721938775510204
AP：  0.6542016810835546
AUC SCORE:  0.6721938775510204
######################### Module cycle ： 51 ##########################
AP：  0.6548701302814155
AUC SCORE:  0.673469387755102
AP：  0.6555303943870578
AUC SCORE:  0.6747448979591837
######################### Module cycle ： 52 ##

AP：  0.7030316733830798
AUC SCORE:  0.7321428571428571
AP：  0.7030316733830798
AUC SCORE:  0.7321428571428571
######################### Module cycle ： 95 ##########################
AP：  0.7030316733830798
AUC SCORE:  0.7321428571428571
AP：  0.7042566407815166
AUC SCORE:  0.7334183673469387
######################### Module cycle ： 96 ##########################
AP：  0.7085703662717127
AUC SCORE:  0.7346938775510204
AP：  0.7084352423267868
AUC SCORE:  0.7334183673469388
######################### Module cycle ： 97 ##########################
AP：  0.709535138807118
AUC SCORE:  0.7346938775510204
AP：  0.709535138807118
AUC SCORE:  0.7346938775510204
######################### Module cycle ： 98 ##########################
AP：  0.7119702037421829
AUC SCORE:  0.735969387755102
AP：  0.7119702037421829
AUC SCORE:  0.735969387755102
######################### Module cycle ： 99 ##########################
AP：  0.7113182763045411
AUC SCORE:  0.7346938775510204
AP：  0.7113182763045411
AUC SCORE:  0.734693

AUC SCORE:  0.7678571428571428
AP：  0.7381445256274473
AUC SCORE:  0.7678571428571428
######################### Module cycle ： 144 ##########################
AP：  0.7381445256274473
AUC SCORE:  0.7678571428571428
AP：  0.7390241878371587
AUC SCORE:  0.7691326530612245
######################### Module cycle ： 145 ##########################
AP：  0.7412601387558155
AUC SCORE:  0.7704081632653063
AP：  0.7421398009655271
AUC SCORE:  0.7716836734693878
######################### Module cycle ： 146 ##########################
AP：  0.7421398009655271
AUC SCORE:  0.7716836734693878
AP：  0.7421398009655271
AUC SCORE:  0.7716836734693878
######################### Module cycle ： 147 ##########################
AP：  0.7421398009655271
AUC SCORE:  0.7716836734693878
AP：  0.7425672325764693
AUC SCORE:  0.7729591836734694
######################### Module cycle ： 148 ##########################
AP：  0.7425672325764693
AUC SCORE:  0.7729591836734694
AP：  0.7425672325764693
AUC SCORE:  0.7729591836734694
####

AUC SCORE:  0.7895408163265306
######################### Module cycle ： 190 ##########################
AP：  0.7620709055578488
AUC SCORE:  0.7895408163265306
AP：  0.7620709055578488
AUC SCORE:  0.7895408163265306
######################### Module cycle ： 191 ##########################
AP：  0.7620709055578488
AUC SCORE:  0.7895408163265306
AP：  0.7620709055578488
AUC SCORE:  0.7895408163265306
######################### Module cycle ： 192 ##########################
AP：  0.7620709055578488
AUC SCORE:  0.7895408163265306
AP：  0.7610269495138928
AUC SCORE:  0.7882653061224489
######################### Module cycle ： 193 ##########################
AP：  0.7610269495138928
AUC SCORE:  0.7882653061224489
AP：  0.7610269495138928
AUC SCORE:  0.7882653061224489
######################### Module cycle ： 194 ##########################
AP：  0.761819000205137
AUC SCORE:  0.7895408163265307
AP：  0.761819000205137
AUC SCORE:  0.7895408163265307
######################### Module cycle ： 195 ################

...

AUC SCORE:  0.7040816326530612
######################### Module cycle ： 79 ##########################
AP：  0.7389572876461007
AUC SCORE:  0.7104591836734694
AP：  0.7383206693624237
AUC SCORE:  0.7091836734693877
######################### Module cycle ： 80 ##########################
AP：  0.7405128154516286
AUC SCORE:  0.7130102040816326
AP：  0.7405128154516286
AUC SCORE:  0.7130102040816326
######################### Module cycle ： 81 ##########################
AP：  0.7352982906173046
AUC SCORE:  0.7091836734693877
AP：  0.7352982906173046
AUC SCORE:  0.7091836734693877
######################### Module cycle ： 82 ##########################
AP：  0.7358825214646932
AUC SCORE:  0.7104591836734693
AP：  0.7358825214646932
AUC SCORE:  0.7104591836734693
######################### Module cycle ： 83 ##########################
AP：  0.7371395541923143
AUC SCORE:  0.7130102040816326
AP：  0.7371395541923143
AUC SCORE:  0.7130102040816326
######################### Module cycle ： 84 ####################

AUC SCORE:  0.7308673469387755
AP：  0.7484792094034387
AUC SCORE:  0.7308673469387755
######################### Module cycle ： 125 ##########################
AP：  0.7503658016471738
AUC SCORE:  0.7334183673469388
AP：  0.7503658016471738
AUC SCORE:  0.7334183673469388
######################### Module cycle ： 126 ##########################
AP：  0.7503658016471738
AUC SCORE:  0.7334183673469388
AP：  0.7503658016471738
AUC SCORE:  0.7334183673469388
######################### Module cycle ： 127 ##########################
AP：  0.750818032245943
AUC SCORE:  0.7334183673469388
AP：  0.750818032245943
AUC SCORE:  0.7334183673469388
######################### Module cycle ： 128 ##########################
AP：  0.7515740806330398
AUC SCORE:  0.7346938775510203
AP：  0.7515740806330398
AUC SCORE:  0.7346938775510203
######################### Module cycle ： 129 ##########################
AP：  0.7515740806330398
AUC SCORE:  0.7346938775510203
AP：  0.7515740806330398
AUC SCORE:  0.7346938775510203
######

AUC SCORE:  0.7423469387755103
######################### Module cycle ： 170 ##########################
AP：  0.7654175004925086
AUC SCORE:  0.7448979591836735
AP：  0.7643839323033282
AUC SCORE:  0.7423469387755102
######################### Module cycle ： 171 ##########################
AP：  0.7664172895058583
AUC SCORE:  0.7448979591836734
AP：  0.7657937067394184
AUC SCORE:  0.7436224489795917
######################### Module cycle ： 172 ##########################
AP：  0.7654166328329326
AUC SCORE:  0.7423469387755102
AP：  0.7654166328329326
AUC SCORE:  0.7423469387755102
######################### Module cycle ： 173 ##########################
AP：  0.7654166328329326
AUC SCORE:  0.7423469387755102
AP：  0.7661726812200294
AUC SCORE:  0.7436224489795918
######################### Module cycle ： 174 ##########################
AP：  0.7661726812200294
AUC SCORE:  0.7436224489795918
AP：  0.7661726812200294
AUC SCORE:  0.7436224489795918
######################### Module cycle ： 175 ##############

AUC SCORE:  0.7474489795918366
AP：  0.7789318331358692
AUC SCORE:  0.7474489795918366
######################### Module cycle ： 218 ##########################
AP：  0.7789318331358692
AUC SCORE:  0.7474489795918366
AP：  0.7789318331358692
AUC SCORE:  0.7474489795918366
######################### Module cycle ： 219 ##########################
AP：  0.7789318331358692
AUC SCORE:  0.7474489795918366
AP：  0.7789318331358692
AUC SCORE:  0.7474489795918366
######################### Module cycle ： 220 ##########################
AP：  0.7801317202053216
AUC SCORE:  0.7487244897959183
AP：  0.7801317202053216
AUC SCORE:  0.7487244897959183
######################### Module cycle ： 221 ##########################
AP：  0.7801317202053216
AUC SCORE:  0.7487244897959183
AP：  0.7801317202053216
AUC SCORE:  0.7487244897959183
######################### Module cycle ： 222 ##########################
AP：  0.7801317202053216
AUC SCORE:  0.7487244897959183
AP：  0.7801317202053216
AUC SCORE:  0.7487244897959183
####

AUC SCORE:  0.7563775510204082
AP：  0.7852792765755527
AUC SCORE:  0.7563775510204082
######################### Module cycle ： 265 ##########################
AP：  0.7852792765755527
AUC SCORE:  0.7563775510204082
AP：  0.7852792765755527
AUC SCORE:  0.7563775510204082
######################### Module cycle ： 266 ##########################
AP：  0.7852792765755527
AUC SCORE:  0.7563775510204082
AP：  0.7852792765755527
AUC SCORE:  0.7563775510204082
######################### Module cycle ： 267 ##########################
AP：  0.7852792765755527
AUC SCORE:  0.7563775510204082
AP：  0.7852792765755527
AUC SCORE:  0.7563775510204082
######################### Module cycle ： 268 ##########################
AP：  0.7852792765755527
AUC SCORE:  0.7563775510204082
AP：  0.7852792765755527
AUC SCORE:  0.7563775510204082
######################### Module cycle ： 269 ##########################
AP：  0.7852792765755527
AUC SCORE:  0.7563775510204082
AP：  0.7852792765755527
AUC SCORE:  0.7563775510204082
####

In [7]:
print("################################  SUM UP #####################################")
print("AP FIRST AP MEAN: ", np.array(all_ap_first_ap).mean())
print("AP FIRST AP STD: ", np.array(all_ap_first_ap).std())

print("AP FIRST AUC MEAN: ", np.array(all_ap_first_auc).mean())
print("AP FIRST AUC STD: ", np.array(all_ap_first_auc).std())

print("AUC FIRST AP MEAN: ", np.array(all_auc_first_ap).mean())
print("AUC FIRST AP STD: ", np.array(all_auc_first_ap).std())

print("AUC FIRST AUC MEAN: ", np.array(all_auc_first_auc).mean())
print("AUC FIRST AUC STD: ", np.array(all_auc_first_auc).std())

################################  SUM UP #####################################
AP FIRST AP MEAN:  0.7739814922215024
AP FIRST AP STD:  0.03667693104854316
AP FIRST AUC MEAN:  0.7604591836734693
AP FIRST AUC STD:  0.037430520112626105
AUC FIRST AP MEAN:  0.760793582981057
AUC FIRST AP STD:  0.04123214513430125
AUC FIRST AUC MEAN:  0.7710459183673469
AUC FIRST AUC STD:  0.03872281764828383
